In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd


In [2]:
browser = webdriver.Edge()

In [10]:
browser.switch_to.window(browser.window_handles[-1])

In [11]:
def go_to_search_page(browser):
    link = "https://www.telecompaper.com/search"
    browser.get(link)
    sleep(5)

In [12]:
def collect_news_links(browser):
    hrefs=[]
    elements_news_link = browser.find_elements(By.XPATH, '//div[@class="column is-12"]/a')

    for element in elements_news_link:
        hrefs.append(element.get_attribute("href"))
    
    print("Total search items: " + str(len(hrefs)))    
    return hrefs

In [13]:
def collect_headlines(browser):
    headline=[]
    elements_news_text = browser.find_elements(By.XPATH, "//div[@class='column is-12']/a/div/div/h2")
    
    for element in elements_news_text:
        headline.append(element.text)
        
    return headline

In [14]:
def extract_news(hrefs,n):
    news = []
    news_date = []
    news_country = []
    news_headline = []
    
    for href in hrefs[:int(n)]:
        browser.get(href)
        sleep(3)

        news_paragraph = browser.find_element(By.XPATH, '//p[@class="is-abstract"]')
        news.append(news_paragraph.text)

        country = browser.find_element(By.XPATH, '//div[@class="is-meta mb-2"]/span[1]')
        news_country.append(country.text)

        date = browser.find_element(By.XPATH, '//div[@class="is-meta mb-2"]/span[2]')
        news_date.append(date.text)
        
        headline = browser.find_element(By.XPATH, '//h2[@class="is-title mb-2"]')
        news_headline.append(headline.text)
    
    
    data = {'Region': news_country, 'Date': news_date, 'Headline':news_headline , 'News': news, 'Link':hrefs[0:len(news)]}
    df = pd.DataFrame(data)
    return df

In [9]:
headers = ['Region' , 'Date', 'Headline', 'News', 'Link']
result_data = pd.DataFrame(columns=headers)

In [15]:
print("Search and add filters.")
y="A"

while 1:
    go_to_search_page(browser)
    y = input('Type "Y" to continue, "E" to exit: ')
    if y=="Y" or y=="y":
        #headlines = collect_headlines(browser)
        news_links = collect_news_links(browser)
        n = input("Out of " + str(len(news_links)) + ", how many news you want to extract: ")
        
        scrapped_data = extract_news(news_links,n)
        result_data = pd.concat([result_data, scrapped_data], ignore_index=True)
        
        
    elif y=="E" or y=="e":
        browser.quit()
        break


    


Search and add filters.
Type "Y" to continue, "E" to exit: y
Total search items: 25
Out of 25, how many news you want to extract: 2
Type "Y" to continue, "E" to exit: y
Total search items: 9
Out of 9, how many news you want to extract: 1
Type "Y" to continue, "E" to exit: e


In [17]:
result_data

,Region,Date,Headline,News,Link
0,GLOBAL,2 FEB,Global enterprise cloud infrastructure service...,Global enterprise spending on cloud infrastruc...,https://www.telecompaper.com/news/global-enter...
1,MIDDLE EAST,31 JAN,Evision to show Disney Star output in Middle East,"Evision, the media and entertainment division ...",https://www.telecompaper.com/news/evision-to-s...
2,POLAND,26 OCT 2023,"Polsat Plus Group, Google Cloud start strategi...",Poland's Polsat Plus Group has launched a stra...,https://www.telecompaper.com/news/polsat-plus-...


In [19]:
result_data.to_excel("data.xlsx")